In [19]:
import requests
import pandas as pd
import json

from dotenv import load_dotenv
import os

load_dotenv()  # defaults to .env in the current dir

True

# Data extraction

In [20]:
TRELLO_API_KEY = os.getenv("TRELLO_API_KEY")
TRELLO_TOKEN = os.getenv("TRELLO_TOKEN")
BOARD_ID = os.getenv("BOARD_ID")


url = f"https://api.trello.com/1/boards/{BOARD_ID}/lists"
params = {"key": TRELLO_API_KEY, "token": TRELLO_TOKEN}

response = requests.get(url, params=params)
lists = response.json()

# Print list names and IDs
for lst in lists:
    print(f"{lst['name']} - {lst['id']}")

INFORMAÇÕES - 61f998f7c3e320873b04e91a
BAÚ DE IDEIAS - 65f2355d10a760b7c7f96bbc
IDEIAS DE VÍDEOS - INSTAGRAM E TIK TOK - 65f23af42884067aab1776c3
PRONTO PARA DESIGN/GRAVAÇÃO - 66fbebab671706c186e560e9
THREADS - 65f2356538cf6755897ae93f
TIK TOK - 66fbeaccb4cb61a8c682ade4
POSTS PRONTOS - 618a55c4c1eb307fcf017a4d
CALENDÁRIO - 66fbeb3c191cf8e62dbfb688
POSTADAS - 65f43cdc87759238f2f07fa8
INSTAGRAM - POSTADAS (ANTIGOS) - 63eff1116f75f425231bc703


In [21]:
LIST_IDs = ["65f43cdc87759238f2f07fa8", "66fbeb3c191cf8e62dbfb688"]


## Just text

In [22]:
def extract_data(LIST_ID, TRELLO_API_KEY, TRELLO_TOKEN):
    url = f"https://api.trello.com/1/lists/{LIST_ID}/cards"
    params = {"key": TRELLO_API_KEY, "token": TRELLO_TOKEN}

    response = requests.get(url, params=params)
    cards = response.json()

    data = []
    for card in cards:

        if card["desc"] == "":
            continue

        data.append({
            "name": card["name"],
            "desc": card["desc"]
        })

    return data

In [23]:
posts_data = []

for LIST_ID in LIST_IDs:
    new_data = extract_data(LIST_ID, TRELLO_API_KEY, TRELLO_TOKEN)
    posts_data.extend(new_data)

# Save to JSON
with open("trello_cards.json", "w", encoding="utf-8") as f:
    json.dump(posts_data, f, ensure_ascii=False, indent=2)

print(f"Saved {len(posts_data)} cards to trello_cards.json")

Saved 71 cards to trello_cards.json


## Also Images

In [24]:
import requests
import os

# Step 1: Get cards in the list
cards_url = f"https://api.trello.com/1/lists/{LIST_ID}/cards"
params = {"key": TRELLO_API_KEY, "token": TRELLO_TOKEN}
cards = requests.get(cards_url, params=params).json()

LIST_ID = '66fbeb3c191cf8e62dbfb688'

# Step 2: Loop through cards and get image attachments
for card in cards:
    print(f"Card: {card['name']}")
    print(f"Text: {card['desc']}")
    
    card_id = card['id']
    attach_url = f"https://api.trello.com/1/cards/{card_id}/attachments"
    attachments = requests.get(attach_url, params=params).json()

    for att in attachments:
        if att.get("isUpload") and att.get("mimeType", "").startswith("image"):
            image_url = att['url']
            print(f"🖼️ Image: {image_url}")

    break


Card: Portfolio AM - Pat
Text: 


# Embeddings

## Generate Embeddings

In [25]:
from sentence_transformers import SentenceTransformer

In [26]:
# Load the JSON file
with open("trello_cards.json", "r", encoding="utf-8") as f:
    posts_data = json.load(f)

# check a few entries
for post in posts_data[:1]:
    print(f"{post['name']}\n{post['desc']}\n")

Eu sou cliente Soeurs, isso significa que…
Entendo a importância do visual & como ele impacta na percepção de valor sobre o meu negócio

Construo diariamente meu posicionamento (dentro e fora do digital), com assertividade & estratégia

Entendo que a a minha marca não é estática, que ela caminha e evolui comigo ao longo do tempo

Amo ter meu universo visual definido & aplicado no meu perfil (e aonde mais quiser)

Conto com um olhar atento & criativo para cada projeto, materializando minhas ideias com muita empolgação

‌

\*trazer bandeiras que levantamos no estúdio



In [27]:
# load model
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Prepare the texts: name + desc
texts = [f"{item['name']}. {item['desc']}" for item in posts_data]

# Generate embeddings
embeddings = model.encode(texts, show_progress_bar=True)

Batches: 100%|██████████| 3/3 [00:00<00:00,  3.52it/s]


## Store Embeddings in FAISS

In [28]:
import faiss
import numpy as np
import json

# Convert to NumPy array (required by FAISS)
embedding_array = np.array(embeddings).astype("float32")

# Get the dimension of each embedding
dimension = embedding_array.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(dimension)  # L2 = Euclidean distance
index.add(embedding_array)

# Save the index
faiss.write_index(index, "posts_index.faiss")

# Save metadata (e.g. post titles, captions)
with open("posts_metadata.json", "w", encoding="utf-8") as f:
    json.dump(posts_data, f, ensure_ascii=False, indent=2)

print(f"Stored {len(posts_data)} embeddings and metadata.")

Stored 71 embeddings and metadata.


# Retrieval + LLM Generation

In [29]:
user_input = 'Escreva uma postagem sobre estratégias para a criação de identidade visual'

In [30]:
query_embedding = model.encode([user_input])[0]  # one vector

In [31]:
import faiss
import numpy as np
import json

# Load index + metadata
index = faiss.read_index("posts_index.faiss")

with open("posts_metadata.json", "r", encoding="utf-8") as f:
    posts_data = json.load(f)

# Search top 3 similar posts
D, I = index.search(np.array([query_embedding]).astype("float32"), k=5)

# Get matched posts
examples = [posts_data[i] for i in I[0]]

In [32]:
examples 

[{'name': '3 mitos sobre identidade visual - FOTO NA CAPA',
  'desc': '3 mitos sobre identidade visual\n\n1- É só a logo ou uma paleta de cores: NÃOOO! Fazemos a construção de todos os itens visuais da sua marca, para que você construa um universo visual estratégico e 100% alinhado com as suas necessidades e desejos.\n\n2- O trabalho acaba após a entrega.\n\nSentimos te informar, mas não. A etapa mais importante vem depois da entrega do manual de marca: aplicação! Precisamos inserir esse universo de marca diariamente, em todos os pontos de contato entre você/seu negócio e o seu cliente.\n\n3- Apenas grandes negócios e marcas corporativas devem ter uma identidade visual\n\nDe forma alguma! Quanto antes você investir nessa parte tão importante da sua empresa, antes conseguirá elevar a percepção de valor em relação ao seu trabalho, atrair clientes alinhados com o seu negócio e construir uma marca memorável.\n\nConta pra gente: Em qual desses mitos você já acreditou?'},
 {'name': 'REELS (r

In [33]:
rag_prompt = "Baseado no estilo dos exemplos abaixo, crie uma nova postagem sobre: " + user_input + "\n\n"

for i, ex in enumerate(examples, 1):
    rag_prompt += f"Exemplo {i}:\nTítulo: {ex['name']}\nLegenda: {ex['desc']}\n\n"

rag_prompt += "Nova legenda:"

In [34]:
rag_prompt

'Baseado no estilo dos exemplos abaixo, crie uma nova postagem sobre: Escreva uma postagem sobre estratégias para a criação de identidade visual\n\nExemplo 1:\nTítulo: 3 mitos sobre identidade visual - FOTO NA CAPA\nLegenda: 3 mitos sobre identidade visual\n\n1- É só a logo ou uma paleta de cores: NÃOOO! Fazemos a construção de todos os itens visuais da sua marca, para que você construa um universo visual estratégico e 100% alinhado com as suas necessidades e desejos.\n\n2- O trabalho acaba após a entrega.\n\nSentimos te informar, mas não. A etapa mais importante vem depois da entrega do manual de marca: aplicação! Precisamos inserir esse universo de marca diariamente, em todos os pontos de contato entre você/seu negócio e o seu cliente.\n\n3- Apenas grandes negócios e marcas corporativas devem ter uma identidade visual\n\nDe forma alguma! Quanto antes você investir nessa parte tão importante da sua empresa, antes conseguirá elevar a percepção de valor em relação ao seu trabalho, atrai

In [35]:
from openai import OpenAI
import os


openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Você é uma criadora de conteúdo dona de um estúdio de design, focada em criação de identidade visual e serviços de artes mensais para o Instagram."},
        {"role": "user", "content": rag_prompt}
    ],
    temperature=0.8,
    max_tokens=300
)

output = response.choices[0].message.content
print("📝 Nova legenda:\n", output)


📝 Nova legenda:
 Título: 5 Estratégias Infalíveis para Criar uma Identidade Visual Impactante

Legenda: CAPA: Estratégias para uma identidade visual de sucesso

PÁGINA 01: Você sabia que uma identidade visual bem construída pode transformar completamente o seu negócio? Vamos explorar 5 estratégias infalíveis para criar uma identidade visual que não só impressiona, mas também conecta com o seu público!

PÁGINA 02: 1. Conheça seu público!

Antes de qualquer criação, é essencial entender quem são as pessoas que você deseja alcançar. Isso vai te ajudar a definir cores, fontes e elementos que realmente ressoam com o seu público-alvo. Lembre-se, o design deve falar diretamente com quem você quer conquistar!

PÁGINA 03: 2. Vá além da logo.

Uma identidade visual completa engloba muito mais do que apenas um logotipo. Pense em paleta de cores, tipografia, ícones, padrões e imagens que, juntos, criam um universo visual coeso e memorável para a sua marca.

PÁGINA 04: 3. Seja consistente.

Manter 